In [0]:
# Dependencies to load moduels from this repo
import importlib.util
import sys

# Load cv module directly from file path
cv_path = "/Workspace/Shared/Team 4_2/flight-departure-delay-predictive-modeling/notebooks/Cross Validator/cv.py"
spec = importlib.util.spec_from_file_location("cv", cv_path)
cv = importlib.util.module_from_spec(spec)
spec.loader.exec_module(cv)

# Dependencies for time series features

# Other Dependencies
import time

# Path for persistent storage
FOLDER_PATH = "dbfs:/mnt/mids-w261/student-groups/Group_4_2/processed"

In [0]:
data_loader = cv.FlightDelayDataLoader()
data_loader.load()
folds = data_loader.get_version("3M")

# Get training data from first fold
train_df, val_df = folds[0]